In [13]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import pickle

df= pd.read_csv("data/titanic_train.csv")
df.columns
df=df.loc[df.Age.notna()&df.Embarked.notna() , ['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked']]
y=df.Survived
X=df.drop("Survived", axis=1)
###############################################################################################

#column transformer
ct= make_column_transformer(
    (OneHotEncoder(), ['Sex','Embarked']),
    remainder='passthrough'
)
#model
logreg= LogisticRegression(solver='lbfgs', max_iter=3000)

## PIPE
pipe= make_pipeline(ct, logreg)

###############################################################################################
skf=StratifiedKFold(5, shuffle=True,random_state=99)
cvs= cross_val_score(pipe,X=X,y=y,cv=skf).mean()
p=pipe.fit(X=X,y=y)

pickle.dump(pipe, open('titanic.pkl','wb'))

In [29]:
X_new=X.sample(5, random_state=1).reset_index().drop("index",axis=1)
X_new.to_csv("data/titanic_test.csv", index=False)

preds=pd.DataFrame(pipe.predict(X_new),columns=["Preds"])
pred_proba= pd.DataFrame(pipe.predict_proba(X_new))
pd.concat([X_new,pred_proba,preds],axis=1)

,Pclass,Sex,Age,Fare,Embarked,0,1
0,1,female,15.0,211.3375,S,0.049394,0.950606
1,3,female,35.0,20.2500,S,0.531588,0.468412
2,3,male,28.0,22.5250,S,0.911504,0.088496
3,2,female,14.0,30.0708,C,0.093793,0.906207
4,1,female,54.0,78.2667,C,0.116118,0.883882
